In [3]:
import pandas as pd 
import gzip

In [4]:
def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l)

In [18]:
def getDF(path): 
    i = 0 
    df = {} 
    for d in parse(path): 
        df[i] = d
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index')

In [19]:
df = getDF('data/reviews_Sports_and_Outdoors_5.json.gz')

In [20]:
df2 = getDF('data/reviews_Video_Games_5.json.gz')

In [25]:
df['type'] = 1
df2['type'] = 2

In [28]:
result = pd.concat([df,df2])

In [52]:
result.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,type
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5.0,Woks very good,1390694400,"01 26, 2014",1
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5.0,Works as well as the factory tool,1328140800,"02 2, 2012",1
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4.0,"It's a punch, that's all.",1330387200,"02 28, 2012",1
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4.0,It's a punch with a Glock logo.,1328400000,"02 5, 2012",1
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4.0,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013",1


In [53]:
result.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,type
231775,A1ICREREXO9J81,B00KHECZXO,Frustrated gamer,"[0, 1]",Funny people on here are rating sellers that a...,5.0,this is for rating the system not the seller,1405814400,"07 20, 2014",2
231776,A3VVMIMMTYQV5F,B00KHECZXO,Johnny Saigon,"[8, 11]",All this is is the Deluxe 32GB Wii U with Mari...,1.0,Get the Other Bundle Which Includes Extra Whee...,1403308800,"06 21, 2014",2
231777,A1DD4B97M4DUC5,B00KHECZXO,migit,"[62, 66]",The package should have more red on it and sho...,1.0,Fake bundle,1401321600,"05 29, 2014",2
231778,A2Q9CNJ4T6ZK99,B00KHECZXO,"Philip Brown ""Philip & Chana""","[33, 36]",Can get this at Newegg for $329.00 and the pac...,1.0,Looks Like We Have Gougers Again.,1401667200,"06 2, 2014",2
231779,A27CD59R5U58GG,B00KHECZXO,ThommyGunn,"[23, 28]","This is not real, you can go to ANY retail sto...",1.0,DO NOT BUY HERE LOOKS TO BE EUROPEAN VERSION,1401840000,"06 4, 2014",2


In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result['reviewText'], result['type'], random_state = 0)
print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry: 

 0    Installing the game was a struggle (because of...
0    This came in on time and I am veru happy with ...
Name: reviewText, dtype: object


X_train shape:  (396087,)


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

236855

In [61]:
X_train_vectorized = vect.transform(X_train)
print(X_train_vectorized.shape)

(396087, 236855)


In [62]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')
model.fit(X_train_vectorized, y_train)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [65]:
import numpy as np
from sklearn.metrics import roc_auc_score
X_test_vectorized = vect.transform(X_test)
y_test_predict = model.predict(X_test_vectorized)
print('AUC: ', roc_auc_score(y_test, y_test_predict))

AUC:  0.9843106374542169


In [67]:
y_train_predict = model.predict(X_train_vectorized)
y_test_predict = model.predict(X_test_vectorized)

err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)

print(err_train, err_test)

0.011868604624741534 0.014413390896008483


In [68]:
pd.crosstab(y_test, y_test_predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,1,2,All
True,,,
1,73660,385,74045
2,1518,56467,57985
All,75178,56852,132030


In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train_vectorized, y_train)

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
X_test_vectorized = vect.transform(X_test)
y_test_predict = model.predict(X_test_vectorized)
print('AUC: ', roc_auc_score(y_test, y_test_predict))

In [ ]:
y_train_predict = model.predict(X_train_vectorized)
y_test_predict = model.predict(X_test_vectorized)

err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)

print(err_train, err_test)

In [ ]:
pd.crosstab(y_test, y_test_predict, rownames=['True'], colnames=['Predicted'], margins=True)